<a href="https://colab.research.google.com/github/profRCC/Softex/blob/master/AgrupamentoMarketingBanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# Importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score

In [4]:
# lendo csv e armazenando em um dataframe
dados = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Softex/Semana4/datasets/marketing/bank.csv')
dados.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
dados = dados.loc[30000:].reset_index(drop=True)

In [6]:
dados[dados.y=='yes'].shape

(3769, 17)

In [7]:
dados[dados.y=='no'].shape

(11442, 17)

In [8]:
#removendo colunas não informativas
dados = dados.drop(columns=['contact','day','month'])
dados.head()

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,poutcome,y
0,40,services,single,secondary,no,4126,no,no,671,2,-1,0,unknown,no
1,38,unemployed,married,tertiary,no,201,no,no,209,1,-1,0,unknown,no
2,32,management,married,tertiary,no,2853,yes,yes,419,1,-1,0,unknown,no
3,39,unemployed,married,secondary,no,1309,yes,no,587,2,-1,0,unknown,no
4,46,blue-collar,single,secondary,no,374,no,yes,23,5,7,2,other,no


In [9]:
jobs = sorted(dados.job.unique())
jobs_num = [i for i in range(len(jobs))]
dados['job'] = dados['job'].replace(jobs,jobs_num)

estado_civil = sorted(dados.marital.unique())
dados.marital = dados.marital.replace(['single', 'married','divorced'],[0,1,2])

educacao = sorted(dados.education.unique())
dados.education = dados.education.replace(['unknown','primary', 'secondary', 'tertiary'],[0,1,2,3])

default = sorted(dados.default.unique())
dados.default = dados.default.replace(['no', 'yes'],[0,1])

housing = sorted(dados.housing.unique())
dados.housing = dados.housing.replace(['no', 'yes'],[0,1])

loan = sorted(dados.loan.unique())
dados.loan = dados.loan.replace(['no', 'yes'],[0,1])

poutcome = dados.poutcome.unique()
dados.poutcome = dados.poutcome.replace(['unknown','other', 'failure', 'success'],[0,1,2,3])

dados.y = dados.y.replace(['no','yes'],[0,1])

dados.head()

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,poutcome,y
0,40,7,0,2,0,4126,0,0,671,2,-1,0,0,0
1,38,10,1,3,0,201,0,0,209,1,-1,0,0,0
2,32,4,1,3,0,2853,1,1,419,1,-1,0,0,0
3,39,10,1,2,0,1309,1,0,587,2,-1,0,0,0
4,46,1,0,2,0,374,0,1,23,5,7,2,1,0


In [10]:
dados = (dados - dados.min())/(dados.max()-dados.min())
dados.head()

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,poutcome,y
0,0.285714,0.636364,0.0,0.666667,0.0,0.077064,0.0,0.0,0.177279,0.045455,0.000000,0.000000,0.000000,0.0
1,0.259740,0.909091,0.5,1.000000,0.0,0.040100,0.0,0.0,0.055218,0.000000,0.000000,0.000000,0.000000,0.0
2,0.181818,0.363636,0.5,1.000000,0.0,0.065076,1.0,1.0,0.110700,0.000000,0.000000,0.000000,0.000000,0.0
3,0.272727,0.909091,0.5,0.666667,0.0,0.050535,1.0,0.0,0.155086,0.045455,0.000000,0.000000,0.000000,0.0
4,0.363636,0.090909,0.0,0.666667,0.0,0.041729,0.0,1.0,0.006077,0.181818,0.009174,0.034483,0.333333,0.0


In [11]:
X = dados.iloc[:,:-1]

In [12]:
y = dados.y

In [13]:
#agrupamento
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [14]:
kmeans_labels = kmeans.predict(X)

In [15]:
hierarquico = AgglomerativeClustering(n_clusters=2)
hierarquico_labels = hierarquico.fit(X).labels_

In [16]:
#índice rand ajustado
rand_score = adjusted_rand_score(y,kmeans_labels)
rand_score

0.1007179818973525

In [17]:
rand_score = adjusted_rand_score(y,hierarquico_labels)
rand_score

0.11267928385596401

In [18]:
#índice baseado em informação mútua
mutual_info = adjusted_mutual_info_score(y,kmeans_labels)
mutual_info

0.0718761774739287

In [19]:
mutual_info = adjusted_mutual_info_score(y,hierarquico_labels)
mutual_info

0.07241085220772646

In [20]:
#índice Fowlkes Mallows
fm = fowlkes_mallows_score(y,kmeans_labels)
fm

0.6094947680929662

In [21]:
fm = fowlkes_mallows_score(y,hierarquico_labels)
fm

0.6182280013417449

In [22]:
#indice Davies-Boldin
db = davies_bouldin_score(X,kmeans_labels)
db

1.4020141054054593

In [23]:
db = davies_bouldin_score(X,hierarquico_labels)
db

1.4421095298461608

In [24]:
#coeficiente de silhueta
ss = silhouette_score(X,kmeans_labels,metric='euclidean')
ss

0.31409962217964726

In [25]:
ss = silhouette_score(X,hierarquico_labels,metric='euclidean')
ss

0.29987529062893364

In [26]:
#indice de Calinski-Harabasz
ch = calinski_harabasz_score(X,kmeans_labels)
ch

7018.98425536219

In [27]:
ch = calinski_harabasz_score(X,hierarquico_labels)
ch

6388.323360486075